## MySQL 접속용 샘플 프로그램

- 접속 정보가 들어있는 파일 읽기

In [7]:
import json

with open('./mysql.json') as f:
    config_str = f.read()
config = json.loads(config_str)
config

{'host': 'localhost',
 'user': 'ysuser',
 'password': 'yspass',
 'database': 'world',
 'port': 3306}

### 2. MySQL에 접속

In [8]:
import pymysql

conn = pymysql.connect(
    host=config['host'],
    user=config['user'],
    password=config['password'],
    database=config['database'],
    port=config['port']
)

### 3. MySQL에서 데이터 읽어오기

- city table에서 10개 데이터 읽어오기

In [12]:
cur = conn.cursor()
sql = "SELECT * FROM city LIMIT 10;"
cur.execute(sql)
rows = cur.fetchall()
cur.close()

for row in rows :
    print(row)

(1, 'Kabul', 'AFG', 'Kabol', 1780000)
(2, 'Qandahar', 'AFG', 'Qandahar', 237500)
(3, 'Herat', 'AFG', 'Herat', 186800)
(4, 'Mazar-e-Sharif', 'AFG', 'Balkh', 127800)
(5, 'Amsterdam', 'NLD', 'Noord-Holland', 731200)
(6, 'Rotterdam', 'NLD', 'Zuid-Holland', 593321)
(7, 'Haag', 'NLD', 'Zuid-Holland', 440900)
(8, 'Utrecht', 'NLD', 'Utrecht', 234323)
(9, 'Eindhoven', 'NLD', 'Noord-Brabant', 201843)
(10, 'Tilburg', 'NLD', 'Noord-Brabant', 193238)


In [16]:
# to dataframe
import pandas as pd

df = pd.DataFrame(rows, columns=['id', 'name', 'country_code', 'district', 'population'])
df.set_index('id', inplace=True)
df

,name,country_code,district,population
id,,,,
1,Kabul,AFG,Kabol,1780000
2,Qandahar,AFG,Qandahar,237500
3,Herat,AFG,Herat,186800
4,Mazar-e-Sharif,AFG,Balkh,127800
5,Amsterdam,NLD,Noord-Holland,731200
6,Rotterdam,NLD,Zuid-Holland,593321
7,Haag,NLD,Zuid-Holland,440900
8,Utrecht,NLD,Utrecht,234323
9,Eindhoven,NLD,Noord-Brabant,201843


- 검색 : 국가코드 KOR의 인구수 Top 10

In [19]:
cur = conn.cursor()
sql = "SELECT * FROM city WHERE countrycode=%s ORDER BY population DESC LIMIT 10;"      # 대부분 ?로 파라미터를 주지만, pymysql에서는 %s를 씀.
cur.execute(sql, ('KOR', ))      # 파라미터 값은 반드시 튜플형태로 주어야 함
rows = cur.fetchall()
cur.close()

In [20]:
for row in rows :
    print(row)

(2331, 'Seoul', 'KOR', 'Seoul', 10000000)
(5003, '대도시', 'KOR', 'Kyonggi', 10000000)
(2332, 'Pusan', 'KOR', 'Pusan', 3804522)
(2333, 'Inchon', 'KOR', 'Inchon', 2559424)
(2334, 'Taegu', 'KOR', 'Taegu', 2548568)
(2335, 'Taejon', 'KOR', 'Taejon', 1425835)
(2336, 'Kwangju', 'KOR', 'Kwangju', 1368341)
(2337, 'Ulsan', 'KOR', 'Kyongsangnam', 1084891)
(2340, '수원', 'KOR', 'Kyonggi', 1000000)
(2338, 'Songnam', 'KOR', 'Kyonggi', 869094)


In [30]:
cur = conn.cursor()
sql = "SELECT * FROM city WHERE name=%s;"
cur.execute(sql, ('수원', ))      
row = cur.fetchone()
cur.close()

In [31]:
row

(2340, '수원', 'KOR', 'Kyonggi', 1000000)

- 삽입 (Insert)

In [21]:
cur = conn.cursor()
sql = "INSERT INTO city VALUES (DEFAULT, 'Java', 'KOR', 'Kyonggi', 6000000);"      # 대부분 ?로 파라미터를 주지만, pymysql에서는 %s를 씀.
cur.execute(sql)      # 파라미터 값은 반드시 튜플형태로 주어야 함
conn.commit()         # DB가 변화되는 SQL(Insert, Update, Delete)에서는 필수
cur.close()

In [24]:
params = ('nodejs', 'KOR', 'Kyonggi', 6000000)
cur = conn.cursor()
sql = "INSERT INTO city VALUES (DEFAULT, %s, %s, %s, %s);"      
cur.execute(sql, params)      
conn.commit()             
cur.close()

In [26]:
params_list = [
    ('HTML', 'KOR', 'Kyonggi', 7000000), ('CSS', 'KOR', 'Kyonggi', 7200000)
]
cur = conn.cursor()
sql = "INSERT INTO city VALUES (DEFAULT, %s, %s, %s, %s);"    
#for params in params_list :
#    cur.execute(sql, params)      
cur.executemany(sql, params_list)       # params가 너무 많으면 실행 오류 발생 가능
conn.commit()             
cur.close()

- 수정 (Update)

In [27]:
params = [5007, 5009]

cur = conn.cursor()
sql = "UPDATE city SET `Name`=%s WHERE id=%s;"      
cur.execute(sql, ('CSS2', 5008))      
conn.commit()        
cur.close()

- 삭제 (Delete)

In [29]:
cur = conn.cursor()
sql = "DELETE FROM city WHERE id>=%s;"      
cur.execute(sql, (5000,))      
conn.commit()        
cur.close()

#### 4. MySQL 접속 해제

In [ ]:
conn.close()